In [34]:
 
import base64
import pandas as pd
from PIL import Image
from openai import OpenAI

client = OpenAI(
    # api_key=".wRDvUO0r3SqRTbrS",
    base_url="https://open.bigmodel.cn/api/paas/v4/"
)
 


def caption_chunk_gpt_compare(video_path, caption_summary_text):
   
    with open(video_path, 'rb') as video_file:
        video_base = base64.b64encode(video_file.read()).decode('utf-8')

    video_description_prompt = f"""视频描述：\"{caption_summary_text}\"

请执行以下任务：

1. 判断视频中是否包含人物信息。

2. 比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

如果视频描述符合上述任务要求，请输出：\"符合\"。

如果视频描述不符合上述任务要求，请输出：\"不符合\"。

请直接输出每个任务结果，不要在回复中提及变量名或额外的信息。
Step by Step Decomposition"""
    print(f"caption_chunk_gpt_compare:{video_description_prompt}")
    # 减少信息，生成速度可以在0.1秒完成
    tools = [
        {
            "type": "web_search",
            "web_search": {
                "enable": False, 
            }
        }
    ]
    response = client.chat.completions.create(
        model="glm-4v-plus",  # 填写需要调用的模型名称
        tools=tools,
        messages=[
          {
            "role": "user",
            "content": [
              {
                "type": "video_url",
                "video_url": {
                    "url" : video_base
                }
              },
              {
                "type": "text",
                "text": video_description_prompt
              }
            ]
          }
        ]
    )

    compare_text = response.choices[0].message.content
    print(f"{video_path}\r\ncaption_chunk_gpt_compare#compare_text:{compare_text}")
    return compare_text


def check_gpt_compare(video_path,compare_text):
    

    video_description_prompt = f"""任务描述：

给定以下文本内容：


{compare_text}


请执行以下操作：

- 判断文本中是否包含关键词 "符合"。
- 如果包含，请输出："是"。
- 如果不包含，请输出："否"。

请直接输出结果，不要包含额外的解释或信息。"""
    print(f"check_gpt_compare:{video_description_prompt}")
    # 减少信息，生成速度可以在0.1秒完成
    tools = [
        {
            "type": "web_search",
            "web_search": {
                "enable": False, 
            }
        }
    ]
    response = client.chat.completions.create(
        model="glm-4-plus",  # 填写需要调用的模型名称
        tools=tools,
        messages=[ 
            {"role": "user", "content": video_description_prompt}
        ]
    )

    check_text = response.choices[0].message.content
    print(f"{video_path}\r\ncheck_gpt_compare#check_text:{check_text}")
    return check_text




def caption_chunk_gpt_role_insert(video_path, features_caption, caption_summary_text):
   
    with open(video_path, 'rb') as video_file:
        video_base = base64.b64encode(video_file.read()).decode('utf-8')

    video_description_prompt = f"""Script Information: {features_caption}

Video Description: "{caption_summary_text}"

Please perform the following tasks:

1. Identify the characters present in both the video and the video description.

2. Compare the video description with the video content and extract the characters from the script to compile a list of roles.

If the video description meets the above requirements, please add the character names to the video description.

If the video description does not meet the above requirements, please output: "Does not meet".

Please output the result directly and do not mention variable names or additional information.

Do not include '\\n' and the word 'video' in your response.

Do not use introductory phrases such as: "The video presents", "The video depicts", "This video showcases", "The video captures", and so on.

Please start the description with the video content directly, such as "A man first sits in a chair, then stands up and walks to the kitchen...."

Do not use phrases like: "as the video progressed" and "Throughout the video".

Please describe the content of the video and the changes that occur, in chronological order.

Please keep the description within 100 English words."""
    print(f"caption_chunk_gpt_role_insert:{video_description_prompt}")
    # 减少信息，生成速度可以在0.1秒完成
    tools = [
        {
            "type": "web_search",
            "web_search": {
                "enable": False, 
            }
        }
    ]
    response = client.chat.completions.create(
        model="glm-4v-plus",  # 填写需要调用的模型名称
        tools=tools,
        messages=[
          {
            "role": "user",
            "content": [
              {
                "type": "video_url",
                "video_url": {
                    "url" : video_base
                }
              },
              {
                "type": "text",
                "text": video_description_prompt
              }
            ]
          }
        ]
    )

    role_caption_text = response.choices[0].message.content
    print(f"{video_path}\r\ncaption_chunk_gpt_role_insert#role_caption_text:{role_caption_text}")
    return role_caption_text


def check_output_folder(output_folder):
    if not os.path.isdir(output_folder):
        print('warning: the folder{} is not exist'.format(output_folder))
        # create srt_folder
        os.makedirs(output_folder)
        print('create folder', output_folder)
       

In [35]:
 
def check_video_caption_exists_role(root_path, video_source):
    save_path = f'{root_path}/{video_source}/' 
    scene_chunks_csv_path = f'{save_path}/scene_chunks/{video_source}_scene_chunk_frame.csv' 
  
    scene_chunks_role_csv_path = f'{save_path}/scene_chunks/{video_source}_scene_chunk_role_frame.csv' 
    
    df = pd.read_csv(scene_chunks_csv_path)
    # Iterate over each row in the table
    for index, row in df.iterrows():
    
        chunk_path = row['chunk_path']
    
        caption_summary_text = row['caption_summary_text']

        compare_text = caption_chunk_gpt_compare(chunk_path,caption_summary_text)
        
        # Assign the compare_text directly to a new column in the DataFrame
        df.at[index, 'role_compare_text'] = compare_text

        check_text = check_gpt_compare(chunk_path, compare_text)
        # Assign the compare_text directly to a new column in the DataFrame
        df.at[index, 'check_text'] = check_text

        if check_text == '是':
            
            features_caption = row['特征描述']
            role_caption_text = caption_chunk_gpt_role_insert(chunk_path, features_caption, caption_summary_text)
            
            df.at[index, 'role_caption_text'] = role_caption_text


    
    df = df.sort_values(by='特征帧', ascending=True)
    df.to_csv(scene_chunks_role_csv_path, index=False)

In [36]:
# check_video_caption_exists_role('/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物/','test_150')


caption_chunk_gpt_compare:视频描述："A mischievous gray cat lounges on a plush green cushion in a cozy living room, reading a book with its tail draped over the cushion's edge. The room features a soft pink wall and a wooden coffee table, creating a tranquil ambiance. At 6 seconds, the cat, now holding a remote control, appears even more relaxed with crossed legs and half-closed eyes. A vibrant yellow pillow and a red wall enhance the domestic setting. The scene transitions from the cat's literary engagement to a moment of contented leisure, emphasizing the cat's central role in this serene domestic tableau."

请执行以下任务：

1. 判断视频中是否包含人物信息。

2. 比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

如果视频描述符合上述任务要求，请输出："符合"。

如果视频描述不符合上述任务要求，请输出："不符合"。

请直接输出每个任务结果，不要在回复中提及变量名或额外的信息。
Step by Step Decomposition
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
caption_chunk_gpt_compare#compare_text:1. 视频描述中未包含人物信息，因此输出："不符合"。

2. 视频描述与视频内容不符，因为视频描述中描述的是一只猫在看书和拿着

/tmp/ipykernel_1772374/613199120.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1. 视频描述中未包含人物信息，因此输出："不符合"。

2. 视频描述与视频内容不符，因为视频描述中描述的是一只猫在看书和拿着遥控器，而视频中并没有出现猫。因此输出："不符合"。
' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'role_compare_text'] = compare_text


/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
check_gpt_compare#check_text:否
caption_chunk_gpt_compare:视频描述："In a dimly lit laboratory, a mischievous brown mouse in a white lab coat transfers green liquid from a large beaker to a smaller container using a green-tinted pipette. The beaker reflects the room's soft light. The mouse's focused expression indicates a scientific experiment. Suddenly, the scene shifts to a bustling city at sunset, showcasing the iconic SIB Tower 12 with reflective glass facades. As the sun sets, the tower's lights twinkle, illuminating the streets. The production company's logo at the tower's base highlights their architectural contribution. The transition from lab to city captures a blend of scientific intrigue and urban sophistication."

请执行以下任务：

1. 判断视频中是否包含人物信息。

2. 比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

如果视频描述符合上述任务要求，请输出："符合"。

如果视频描述不符合上述任务要求，请输出："不符合"。

请直接输出每个任务结果，不要在回复中提及变量名或额外的信息。
Step by Step 

/tmp/ipykernel_1772374/613199120.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '否' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'check_text'] = check_text


/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
caption_chunk_gpt_compare#compare_text:任务1：判断视频中是否包含人物信息。

视频描述中并未提及任何人物信息，因此视频中没有包含人物信息。

任务2：比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

视频描述中提到了一个实验室的场景，以及一只穿着白色 lab coat 的棕色老鼠正在用绿色 pipette 将绿色液体从大烧杯转移到小烧杯中。此外，视频描述还提到了一个城市日落时的场景，以及一座名为 SIB Tower 12 的摩天大楼。然而，视频内容中并没有出现任何实验室或城市的场景，而是展示了一只棕色老鼠在日落时分的城市街道上奔跑。因此，视频描述并未准确反映视频的内容。

综上所述，视频描述不符合视频内容。
check_gpt_compare:任务描述：

给定以下文本内容：


任务1：判断视频中是否包含人物信息。

视频描述中并未提及任何人物信息，因此视频中没有包含人物信息。

任务2：比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

视频描述中提到了一个实验室的场景，以及一只穿着白色 lab coat 的棕色老鼠正在用绿色 pipette 将绿色液体从大烧杯转移到小烧杯中。此外，视频描述还提到了一个城市日落时的场景，以及一座名为 SIB Tower 12 的摩天大楼。然而，视频内容中并没有出现任何实验室或城市的场景，而是展示了一只棕色老鼠在日落时分的城市街道上奔跑。因此，视频描述并未准确反映视频的内容。

综上所述，视频描述不符合视频内容。


请执行以下操作：

- 判断文本中是否包含关键词 "符合"。
- 如果包含，请输出："是"。
- 如果不包含，请输出："否"。

请直接输出结果，不要包含额外的解释或信息。
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
check_gpt_compare#check_tex

/tmp/ipykernel_1772374/613199120.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A cartoon mouse is seen in motion, its body angled forward with limbs stretched out, suggesting swift movement against a simple backdrop with a large white shape and a green ground. As the mouse continues its journey, it is captured mid-stride with a look of surprise or excitement, against a background of a blue sky and green ground, indicating ongoing movement. The scene shifts to show the mouse in a dynamic pose, with one leg lifted and arms outstretched, against a backdrop of a blue curtain and green carpet, suggesting rapid movement within an indoor setting.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'role_caption_text'] = role_caption_text


/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
caption_chunk_gpt_compare#compare_text:任务1：判断视频中是否包含人物信息。

视频描述中没有提到任何人物信息，因此视频描述中没有包含人物信息。

任务2：比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

视频描述中提到了一个 whimsical landscape，一个 mischievous gray cat，一个 red and yellow bird，以及一个 soft pastel background。这些元素在视频中都可以看到，因此视频描述准确反映了视频的内容。

因此，视频描述符合上述任务要求，输出为："符合"。
check_gpt_compare:任务描述：

给定以下文本内容：


任务1：判断视频中是否包含人物信息。

视频描述中没有提到任何人物信息，因此视频描述中没有包含人物信息。

任务2：比较视频描述与视频内容，判断视频描述是否准确反映了视频的内容。

视频描述中提到了一个 whimsical landscape，一个 mischievous gray cat，一个 red and yellow bird，以及一个 soft pastel background。这些元素在视频中都可以看到，因此视频描述准确反映了视频的内容。

因此，视频描述符合上述任务要求，输出为："符合"。


请执行以下操作：

- 判断文本中是否包含关键词 "符合"。
- 如果包含，请输出："是"。
- 如果不包含，请输出："否"。

请直接输出结果，不要包含额外的解释或信息。
/mnt/ceph/develop/jiawei/lora_dataset/speech_data/猫和老鼠_只有人物//test_150//scene_chunks/videos/chunk_653_1.mp4
check_gpt_compare#check_text:是
caption_chunk_gpt_role_insert:Script Information: In the heart of a dimly lit 

KeyboardInterrupt: 

In [ ]:
import os

root_path = '/mnt/ceph/develop/jiawei/lora_dataset/speech_data/B站/'
for root, dirs, files in os.walk(root_path):
    # 如果你只想获取下一层的子目录，可以在这里筛选
    if root == root_path:
        # root_dir 下的直接子目录就是 dirs 中的项
        for dir in dirs:
            check_video_caption_exists_role(root_path,dir)
    